# Model Creation Notebook

This notebook, we will cover the creation of machine learning models for prediction.

## Import Libraries and Datasets

In [12]:
import pandas as pd
pd.pandas.set_option('display.max_columns', None)

import numpy as np 

import warnings
warnings.filterwarnings('ignore')

import joblib

In [13]:
# Load the data
train_path = '/Users/manueljohn/Training/github-projects/bike-demand-prediction/artifacts/raw_data/train_data.csv'
val_path = '/Users/manueljohn/Training/github-projects/bike-demand-prediction/artifacts/raw_data/validation_data.csv'
test_path = '/Users/manueljohn/Training/github-projects/bike-demand-prediction/artifacts/raw_data/test_data.csv'

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

train_df.shape, val_df.shape, test_df.shape

((7008, 14), (876, 14), (876, 14))

In [14]:
# Import the pipeline
pre_processor_path = "/Users/manueljohn/Training/github-projects/bike-demand-prediction/artifacts/pipeline-components/preprocessing_pipeline.pkl"

pre_preocessing_pipeline = joblib.load(pre_processor_path)
pre_preocessing_pipeline

Pipeline(steps=[('cleaning_pipeline',
                 Pipeline(steps=[('clean_col_transformer',
                                  FunctionTransformer(func=<function clean_col_names at 0x7fe9427e4040>)),
                                 ('imputer', NullValueImputer())])),
                ('feature_transform_pipeline',
                 Pipeline(steps=[('skew_discretizer', SkewDiscretizer()),
                                 ('multicollinear_transformer',
                                  FunctionTransformer(func=<function remove_multicollinear_features at 0x7fe9617a8a60>)),
                                 ('categorical_encoder', CategoricalEncoder()),
                                 ('date_features_transformer',
                                  FunctionTransformer(func=<function extract_date_features at 0x7fe9427cd040>)),
                                 ('lag_features_transformer',
                                  LagFeatureCreator())]))])